In [1]:
pip install pandas numpy linearmodels statsmodels openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
#誤差修正模型(ECM)計算長短期影響
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_excel('final_efficiency_data_final2.xlsx')
df['lnCE'] = np.where(df['CE'] > 0, np.log(df['CE']), np.nan)
df['lnLI'] = np.where(df['LI'] > 0, np.log(df['LI']), np.nan)

df = df.sort_values(['id', 'year']).set_index(['id', 'year'])
panel = df[['lnCE', 'lnLI']].dropna()

panel['lnLI_L1'] = panel.groupby(level=0)['lnLI'].shift(1)
panel['lnCE_L1'] = panel.groupby(level=0)['lnCE'].shift(1)
panel['dlnLI']   = panel.groupby(level=0)['lnLI'].diff()
panel['dlnCE']   = panel.groupby(level=0)['lnCE'].diff()

ecm = panel.dropna(subset=['lnLI_L1', 'lnCE_L1', 'dlnLI', 'dlnCE'])
X = ecm[['lnLI_L1', 'lnCE_L1', 'dlnCE']]
X = sm.add_constant(X)
y = ecm['dlnLI']

model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': ecm.index.get_level_values(0)})
print(model.summary())

# 長期係數
phi = -model.params['lnCE_L1'] / model.params['lnLI_L1']
print('\n[手動 ECM] 長期 φ =', phi)

                            OLS Regression Results                            
Dep. Variable:                  dlnLI   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     14.95
Date:                Thu, 03 Jul 2025   Prob (F-statistic):           0.000235
Time:                        02:03:04   Log-Likelihood:                -111.30
No. Observations:                 181   AIC:                             230.6
Df Residuals:                     177   BIC:                             243.4
Df Model:                           3                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2462      0.103     -2.400      0.0

C:\Users\User\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
import pandas as pd
import numpy as np

# 請將路徑改成你目前分析的資料檔
df = pd.read_excel('final_efficiency_data_final2.xlsx')
df['lnCE'] = np.where(df['CE'] > 0, np.log(df['CE']), np.nan)
df = df.sort_values(['id', 'year']).set_index(['id', 'year'])
panel = df[['lnCE']].dropna()

# 一階差分
panel['dlnCE'] = panel.groupby(level=0)['lnCE'].diff()

# 波動度摘要
print('【lnCE】')
print(panel['lnCE'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))
print('\n【dlnCE（一階差分）】')
print(panel['dlnCE'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))

【lnCE】
count    209.000000
mean      -1.152799
std        0.547956
min       -1.999295
1%        -1.993554
5%        -1.970754
25%       -1.527240
50%       -1.278290
75%       -0.840439
95%       -0.176388
99%       -0.174348
max       -0.173842
Name: lnCE, dtype: float64

【dlnCE（一階差分）】
count    194.000000
mean      -0.001628
std        0.000811
min       -0.004884
1%        -0.003540
5%        -0.002737
25%       -0.002114
50%       -0.001771
75%       -0.001184
95%       -0.000246
99%       -0.000244
max       -0.000243
Name: dlnCE, dtype: float64
